In [36]:
import pandas as pd
import numpy as np
import argparse
import sys
import os

infp="./hisat_to_ref.annotated.gtf"

gff3cols=["seqid","source","type","start","end","score","strand","phase","attributes"]
df=pd.read_csv(infp,sep="\t",names=gff3cols,comment="#")
df=df[df["type"]=="exon"].reset_index(drop=True)
df.head()

,seqid,source,type,start,end,score,strand,phase,attributes
0,chr1,StringTie,exon,12012,12213,.,+,.,"transcript_id ""TCONS_00008836""; gene_id ""XLOC_..."
1,chr1,StringTie,exon,69091,70008,.,+,.,"transcript_id ""TCONS_00000001""; gene_id ""XLOC_..."
2,chr1,StringTie,exon,183635,184238,.,+,.,"transcript_id ""TCONS_00006031""; gene_id ""XLOC_..."
3,chr1,StringTie,exon,183635,184428,.,+,.,"transcript_id ""TCONS_00008837""; gene_id ""XLOC_..."
4,chr1,StringTie,exon,597422,597636,.,+,.,"transcript_id ""TCONS_00021307""; gene_id ""XLOC_..."


In [37]:
df["parent"]=df["attributes"].str.split("transcript_id \"",expand=True)[1].str.split("\";",expand=True)[0]
# df["parent"]=df["attributes"].str.split("Parent=",expand=True)[1].str.split(";",expand=True)[0]
df.head()

,seqid,source,type,start,end,score,strand,phase,attributes,parent
0,chr1,StringTie,exon,12012,12213,.,+,.,"transcript_id ""TCONS_00008836""; gene_id ""XLOC_...",TCONS_00008836
1,chr1,StringTie,exon,69091,70008,.,+,.,"transcript_id ""TCONS_00000001""; gene_id ""XLOC_...",TCONS_00000001
2,chr1,StringTie,exon,183635,184238,.,+,.,"transcript_id ""TCONS_00006031""; gene_id ""XLOC_...",TCONS_00006031
3,chr1,StringTie,exon,183635,184428,.,+,.,"transcript_id ""TCONS_00008837""; gene_id ""XLOC_...",TCONS_00008837
4,chr1,StringTie,exon,597422,597636,.,+,.,"transcript_id ""TCONS_00021307""; gene_id ""XLOC_...",TCONS_00021307


In [38]:
df.sort_values(by=["parent","seqid","strand","start","end"],ascending=True,inplace=True)
df["next_parent"]=df["parent"].shift(-1)
df["next_start"]=df["start"].shift(-1)
df.dropna(inplace=True)
df["next_start"]=df.next_start.astype(int)
df=df[df["parent"]==df["next_parent"]].reset_index(drop=True)

df["start"]=df["end"]
df["end"]=df["next_start"]
df["type"]="intron"
df.drop_duplicates(["seqid","strand","start","end"],inplace=True)

In [39]:
df.drop(["next_start","parent","next_parent"],axis=1,inplace=True)
df.head()

,seqid,source,type,start,end,score,strand,phase,attributes
0,chr1,StringTie,intron,735549,738961,.,+,.,"transcript_id ""TCONS_00000005""; gene_id ""XLOC_..."
1,chr1,StringTie,intron,818202,818723,.,+,.,"transcript_id ""TCONS_00000007""; gene_id ""XLOC_..."
2,chr1,StringTie,intron,827775,829003,.,+,.,"transcript_id ""TCONS_00000009""; gene_id ""XLOC_..."
3,chr1,StringTie,intron,829104,841200,.,+,.,"transcript_id ""TCONS_00000009""; gene_id ""XLOC_..."
5,chr1,StringTie,intron,829104,847654,.,+,.,"transcript_id ""TCONS_00000010""; gene_id ""XLOC_..."


In [40]:
df.to_csv("hisat_to_ref.intron.gff",sep="\t",header=False,index=False)